In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Dense, Dropout, BatchNormalization,
    GlobalAveragePooling2D, RandomFlip, RandomRotation, RandomZoom, RandomContrast,
    LSTM, Reshape, Input
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard, CSVLogger
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.utils import compute_class_weight
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime
import time
from PIL import Image
import logging

# Suppress TensorFlow warnings for cleaner output
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Enable XLA for optimization
tf.config.optimizer.set_jit(True)

# Set seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Enable mixed precision
from tensorflow.keras.mixed_precision import set_global_policy
set_global_policy('mixed_float16')

# Paths and hyperparameters
CSV_PATH = r"C:\Users\ahmed\Downloads\ML-Project\Dataset\image_labels.csv"
IMG_DIR = r"C:\Users\ahmed\Downloads\ML-Project\Dataset\interior"
IMG_HEIGHT, IMG_WIDTH = 224, 224
BATCH_SIZE = 32  # Reduced for better generalization
EPOCHS = 200

def initialize_environment():
    """Log environment details and initialize settings."""
    print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

def regenerate_csv(image_dir, output_file):
    """
    Scans the image directory, matches filenames to class labels based on predefined variations,
    and saves the mapping to a CSV file.
    
    Args:
        image_dir (str): Directory containing the images.
        output_file (str): Path to save the generated CSV file.
    
    Returns:
        pd.DataFrame: DataFrame containing image paths and labels.
    """
    classes = ['bath', 'bed', 'dining room', 'kitchen', 'living room']
    class_variations = {
        'bath': ['bath', 'bathroom'], 'bed': ['bed', 'bedroom'],
        'dining room': ['dining', 'dining_room', 'diningroom', 'din'],
        'kitchen': ['kitchen'], 'living room': ['living', 'living_room', 'livingroom']
    }
    data = []
    for filename in os.listdir(image_dir):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            matched = False
            for cls in classes:
                for variation in class_variations[cls]:
                    if variation.lower() in filename.lower():
                        img_path = os.path.abspath(os.path.join(image_dir, filename))
                        data.append({'image_path': img_path, 'label': cls})
                        matched = True
                        break
                if matched:
                    break
    df = pd.DataFrame(data)
    df.to_csv(output_file, index=False)
    logging.info(f"Regenerated CSV with {len(df)} images")
    return df

def validate_images(df, img_dir):
    """
    Validate image paths and log errors for invalid images.
    
    Args:
        df (pd.DataFrame): DataFrame containing image paths and labels.
        img_dir (str): Directory containing the images.
    
    Returns:
        list: List of invalid image paths.
    """
    invalid_images = []
    for img_path in df['image_path']:
        img_path = img_path.replace('/', '\\')  # Normalize for Windows
        if not os.path.exists(img_path):
            invalid_images.append(img_path)
            logging.error(f"Image not found: {img_path}")
            continue
        try:
            with Image.open(img_path) as img:
                img.verify()
        except Exception as e:
            invalid_images.append(img_path)
            logging.error(f"Invalid image {img_path}: {e}")
    return invalid_images

def verify_data(csv_path, img_dir):
    """
    Verify that all image paths in the CSV file exist.
    
    Args:
        csv_path (str): Path to the CSV file.
        img_dir (str): Directory containing the images.
    
    Raises:
        ValueError: If any image file is missing.
    """
    df = pd.read_csv(csv_path)
    print(f"Total images: {len(df)}, Classes: {df['label'].value_counts()}")
    missing = [path for path in df['image_path'] if not os.path.exists(path)]
    if missing:
        raise ValueError(f"Missing files: {missing}")

def load_and_preprocess_data(csv_path, img_dir):
    """
    Load and preprocess the dataset with error handling.
    
    Args:
        csv_path (str): Path to the CSV file.
        img_dir (str): Directory containing the images.
    
    Returns:
        tuple: (train_dataset, val_dataset, num_classes, label_encoder, val_df, class_weight_dict)
    """
    df = pd.read_csv(csv_path)
    label_encoder = LabelEncoder()
    df['label_encoded'] = label_encoder.fit_transform(df['label'])
    num_classes = len(label_encoder.classes_)

    train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)
    class_weights = compute_class_weight('balanced', classes=np.unique(df['label_encoded']), y=df['label_encoded'])
    class_weight_dict = dict(enumerate(class_weights))

    def load_image(image_path, label):
        try:
            img = tf.io.read_file(image_path)
            img = tf.cond(tf.strings.length(img) == 0,
                          lambda: tf.zeros([IMG_HEIGHT, IMG_WIDTH, 3], tf.float32),
                          lambda: tf.image.resize(tf.image.decode_jpeg(img, channels=3), [IMG_HEIGHT, IMG_WIDTH]))
            img = tf.cast(img, tf.float32)
            return img, label
        except tf.errors.InvalidArgumentError as e:
            logging.error(f"Failed to load image {image_path}: {e}")
            return tf.zeros([IMG_HEIGHT, IMG_WIDTH, 3], tf.float32), label

    def filter_valid_samples(image, label):
        return tf.reduce_any(tf.not_equal(image, tf.zeros([IMG_HEIGHT, IMG_WIDTH, 3])))

    train_dataset = tf.data.Dataset.from_tensor_slices(
        (train_df['image_path'].values, tf.keras.utils.to_categorical(train_df['label_encoded'], num_classes))
    ).map(load_image, num_parallel_calls=tf.data.AUTOTUNE).filter(filter_valid_samples).cache().shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    val_dataset = tf.data.Dataset.from_tensor_slices(
        (val_df['image_path'].values, tf.keras.utils.to_categorical(val_df['label_encoded'], num_classes))
    ).map(load_image, num_parallel_calls=tf.data.AUTOTUNE).filter(filter_valid_samples).cache().batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    return train_dataset, val_dataset, num_classes, label_encoder, val_df, class_weight_dict

def build_cnn_lstm_model(num_classes):
    """
    Build a CNN-LSTM model.
    
    Args:
        num_classes (int): Number of output classes.
    
    Returns:
        tf.keras.Model: Compiled CNN-LSTM model.
    """
    input_layer = Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
    x = input_layer
    x = Conv2D(32, (3, 3), padding='same', activation='relu', kernel_regularizer=l2(0.005))(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)
    x = Dropout(0.5)(x)
    x = Conv2D(64, (3, 3), padding='same', activation='relu', kernel_regularizer=l2(0.005))(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)
    x = Dropout(0.5)(x)
    x = Conv2D(128, (3, 3), padding='same', activation='relu', kernel_regularizer=l2(0.005))(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)
    x = Dropout(0.5)(x)
    x = Conv2D(256, (3, 3), padding='same', activation='relu', kernel_regularizer=l2(0.005))(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)
    x = Dropout(0.5)(x)
    x = Conv2D(256, (3, 3), padding='same', activation='relu', kernel_regularizer=l2(0.005))(x)
    x = BatchNormalization()(x)
    x = Conv2D(256, (3, 3), padding='same', activation='relu', kernel_regularizer=l2(0.005))(x)
    x = BatchNormalization()(x)
    x = GlobalAveragePooling2D()(x)
    x = Reshape((1, 256))(x)
    x = LSTM(512, return_sequences=False, dropout=0.3, recurrent_dropout=0.2)(x)
    x = Dense(64, activation='relu', kernel_regularizer=l2(0.01))(x)
    x = Dropout(0.65)(x)
    output = Dense(num_classes, activation='softmax', dtype='float32')(x)

    model = Model(inputs=input_layer, outputs=output)
    return model

def compile_and_train_model(model, train_dataset, val_dataset, num_classes, class_weight_dict):
    """
    Compile and train the CNN-LSTM model with specified callbacks.
    
    Args:
        model: The CNN-LSTM model to train.
        train_dataset: Training dataset.
        val_dataset: Validation dataset.
        num_classes (int): Number of output classes.
        class_weight_dict (dict): Class weights for training.
    
    Returns:
        tuple: (model, history)
    """
    learning_rate = 5e-4
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    model.build((None, IMG_HEIGHT, IMG_WIDTH, 3))
    model.summary()

    class TimeHistory(tf.keras.callbacks.Callback):
        def on_train_begin(self, logs={}):
            self.times = []
        def on_epoch_begin(self, epoch, logs={}):
            self.epoch_time_start = time.time()
        def on_epoch_end(self, epoch, logs={}):
            self.times.append(time.time() - self.epoch_time_start)

    callbacks = [
        EarlyStopping(monitor='val_accuracy', patience=15, restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=7, min_lr=1e-6),
        TensorBoard(log_dir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")),
        CSVLogger('training_log_cnn_lstm.csv', append=True),
        TimeHistory()
    ]

    history = model.fit(
        train_dataset, epochs=EPOCHS, validation_data=val_dataset,
        callbacks=callbacks, class_weight=class_weight_dict, verbose=1
    )
    return model, history

def evaluate_model(model, val_dataset):
    """
    Evaluate the model on the validation set.
    
    Args:
        model: Trained model.
        val_dataset: Validation dataset.
    
    Returns:
        tuple: (val_loss, val_accuracy)
    """
    val_loss, val_accuracy = model.evaluate(val_dataset)
    print(f"Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}")
    return val_loss, val_accuracy

def save_model_and_metadata(model, label_encoder, history_callback):
    """
    Save the trained model and related metadata.
    
    Args:
        model: Trained model.
        label_encoder: Label encoder object.
        history_callback: Callback containing epoch times.
    """
    model.save('model_cnn_lstm.keras')
    np.save('label_encoder_classes_cnn_lstm.npy', label_encoder.classes_)

    with open('epoch_times_cnn_lstm.txt', 'w') as f:
        f.write("Epoch Times (seconds):\n" + "\n".join(map(str, history_callback.times)))

def plot_training_history(history):
    """
    Plot and save training history.
    
    Args:
        history: Training history object.
    """
    plt.figure(figsize=(12, 4))
    for metric in ['accuracy', 'loss']:
        plt.subplot(1, 2, 1 if metric == 'accuracy' else 2)
        plt.plot(history.history[metric], label=f'Training {metric.capitalize()}')
        plt.plot(history.history[f'val_{metric}'], label=f'Validation {metric.capitalize()}')
        plt.title(f'Training and Validation {metric.capitalize()}')
        plt.legend()
    plt.savefig('training_history_cnn_lstm.png')
    plt.close()

def generate_and_evaluate_predictions(model, val_dataset, label_encoder, val_df):
    """
    Generate predictions and evaluate model performance.
    
    Args:
        model: Trained model.
        val_dataset: Validation dataset.
        label_encoder: Label encoder object.
        val_df: Validation DataFrame.
    """
    val_images, val_labels = next(iter(val_dataset))
    predictions = model.predict(val_images)
    predicted_labels = label_encoder.inverse_transform(np.argmax(predictions, axis=1))
    true_labels = label_encoder.inverse_transform(np.argmax(val_labels, axis=1))
    print("Sample Predictions:", *[(t, p) for t, p in zip(true_labels[:10], predicted_labels[:10])], sep='\n')

    val_predictions = model.predict(val_dataset)
    val_pred_labels = np.argmax(val_predictions, axis=1)
    val_true_labels = np.argmax(np.concatenate([y for _, y in val_dataset]), axis=1)
    report = classification_report(val_true_labels, val_pred_labels, target_names=label_encoder.classes_)
    print("\nClassification Report:\n", report)

    with open('classification_report_cnn_lstm.txt', 'w') as f:
        f.write(report)

    plot_confusion_matrix(val_true_labels, val_pred_labels, label_encoder.classes_)

    val_images_paths = val_df['image_path'].values
    misclassified = [(t, p, img) for t, p, img in zip(label_encoder.inverse_transform(val_true_labels),
                                                     label_encoder.inverse_transform(val_pred_labels),
                                                     val_images_paths) if t != p]
    print(f"Misclassified examples: {len(misclassified)}")
    with open('misclassified_images_cnn_lstm.txt', 'w') as f:
        for t, p, img in misclassified[:10]:
            f.write(f"True: {t}, Predicted: {p}, Image: {img}\n")

def main():
    """
    Execute the full CNN-LSTM pipeline.
    """
    # Step 1: Initialize environment
    initialize_environment()

    # Step 2: Regenerate CSV
    df = regenerate_csv(IMG_DIR, CSV_PATH)
    if df is None:
        return

    # Step 3: Validate images
    invalid_images = validate_images(df, IMG_DIR)
    if invalid_images:
        logging.warning(f"Found {len(invalid_images)} invalid images. Removing them.")
        df = df[~df['image_path'].isin(invalid_images)]
        df.to_csv(CSV_PATH, index=False)
        print(f"Updated CSV with {len(df)} valid images")

    # Step 4: Verify data
    verify_data(CSV_PATH, IMG_DIR)

    # Step 5: Load and preprocess data
    train_dataset, val_dataset, num_classes, label_encoder, val_df, class_weight_dict = load_and_preprocess_data(CSV_PATH, IMG_DIR)

    # Step 6: Build and train model
    model = build_cnn_lstm_model(num_classes)
    model, history = compile_and_train_model(model, train_dataset, val_dataset, num_classes, class_weight_dict)

    # Step 7: Evaluate model
    evaluate_model(model, val_dataset)

    # Step 8: Save model and metadata
    save_model_and_metadata(model, label_encoder, history.callbacks[-1])

    # Step 9: Plot training history
    plot_training_history(history)

    # Step 10: Generate and evaluate predictions
    generate_and_evaluate_predictions(model, val_dataset, label_encoder, val_df)

if __name__ == "__main__":
    main()

Validation Accuracy: 0.8979

Classification Report:
               precision    recall  f1-score   support

        bath       0.93      0.95      0.94       486
         bed       0.92      0.91      0.91       489
 dining room       0.90      0.88      0.89       521
     kitchen       0.92      0.87      0.89       447
 living room       0.83      0.88      0.86       524

    accuracy                           0.90      2467
   macro avg       0.90      0.90      0.90      2467
weighted avg       0.90      0.90      0.90      2467

SVM model saved as 'svm_model.pkl'
